In [ ]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import time


<h2>Scraping Amazon product info</h2>
<p>Have written a function to scrape all the details of a product link like:<br>->Name of product<br>->Category<br>->Description<br>->Review<br>->Rating<br>Make sure to only paste the product link.

In [ ]:
#-------------To scrape any product individually use this function-------------------------#

def scrape(url, filename):
    ''' Parameters: Url to be scraped, filename to append the data'''
    temp_Product_info={
    'name': [],
    'category': [],
    'Description': [],
    'Review': [],
    'Rating': []
   }
    i=0
    df_temp=pd.DataFrame(temp_Product_info)
    proxy_temp={
    'api_key': #Place your API key here (Scraperapi),
    'url': url
    }
    print("\n\n Fetching details of the new product...\n")

    product=requests.get('http://api.scraperapi.com', params=proxy_temp)
    if product.status_code==200:
        product=BeautifulSoup(product.content, 'html.parser')

        prod_title=(product.find('span', id="productTitle")).text 
        
        product_desc=product.find_all('li', class_="a-spacing-mini") #product description
        product_desc[1]=product_desc[1].text
        product_desc[0]=product_desc[0].text
        product_description=product_desc[0]+product_desc[1]


        reviews_link=(product.find('a', class_="a-link-emphasis a-text-bold"))['href']  
        reviews_link="amazon.in"+reviews_link

        #inside the review page

        proxy2={
            'api_key': 'bcf253a2f3702ac6d9df7bfdb114c216',
            'url': reviews_link
        }

        print("Waiting to fetch review_page... \n ")
        time.sleep(45)
        

        reviews=requests.get('http://api.scraperapi.com', params=proxy2)

        if reviews.status_code==200:
            
            print("Fetched\n")
            reviews=BeautifulSoup(reviews.content, 'html.parser')
            
            reviews =reviews.find_all('div', class_="a-section review aok-relative")
            for review in reviews:
                rating=review.find('span', class_="a-icon-alt").text
                rating=rating[0:3]
                
                para=review.find('span', class_="a-size-base review-text review-text-content").text
                

                tuple={
                    'name': prod_title,
                    'category': category,
                    'Description': product_description,
                    'Review': para,
                    'Rating': rating
                }

                df_temp=df_temp.append(tuple, ignore_index=True)

                print(f"finished Appending review  {i}")
                i+=1
                    
        else:
            print(f"Unable to fetch review page  \n")
            print(reviews.status_code)
            
            

    
    else:
        print(product.status_code)
        print(f"connection interrupted")
        
    df_temp.to_csv(filename, mode='a') ## To write to a csv file

In [ ]:
link=''# Paste your link here
filename='' #paste the file name you wish to append the information to.
scrape(link, filename)